In [2]:
import os 
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
from pydantic import BaseModel,Field
from typing import Optional,Literal
from typing import TypedDict
load_dotenv()



GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")


c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
Model = ChatGoogleGenerativeAI(model='gemini-2.5-flash',api_key=GOOGLE_API_KEY,temperature=.5)
evaluator_llm =  ChatGoogleGenerativeAI(model='gemini-2.5-flash',api_key=GOOGLE_API_KEY,temperature=0)
optimizer_llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash',api_key=GOOGLE_API_KEY,temperature=.7)


In [4]:
Model.invoke("what is 2+2 and multiply with 5").content

"Let's break it down:\n\n1.  **First, calculate 2 + 2:**\n    2 + 2 = 4\n\n2.  **Then, multiply that result by 5:**\n    4 * 5 = 20\n\nSo, the answer is **20**."

In [10]:
from langgraph.graph import StateGraph,START,END
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage

In [ ]:
# Lets the dicuss the workflow.

# 1: start
# 2: genration
# 3: evaluate 
# 4: if need improvemenet then # Optimize 
# Approved End

In [ ]:
#state 
# topic:str
# genrate the topic on the post 
# ealuate if need improvement then optimize node else end 

In [5]:
# State
class TweetSatate(TypedDict):
    topic:str
    tweet:str
    evaluation : Literal["approved",'need_improvement']
    feedback:str
    iteration:int
    max_iteration:int

In [8]:
graph = StateGraph(TweetSatate)

In [12]:
def genrate_tweet(state:TweetSatate):
   message = [
      SystemMessage(content='you are a funny and cleaver tweet/x writing influncer.'),
      HumanMessage(content=f"""
      you are a helpfull assistant to write the tweet on the given topic.
        tweet should be clear,concise and according to the topic. if there is no
        information or user may asking that you dont know just simply mentioned 
        i dont have idea about this thing. I tweet tried to mention the words that 
        are easy to read and understand ,dont add to much heavy words ,which cant
        understand by the poeple.Max character should be arround 300.
      topic is - {state['tweet']}
      """)
   ]
   response=Model.invoke(message).content

   return {'tweet':response}

In [18]:
class EvaluationSchema(BaseModel):
    evaluation:Literal["approved",'need_improvement']
    feedback:str = Field(description="constructive feedback for the tweet")
    score :float = Field(description="total score out of 0 to 5",gt=0,lt=5)

In [ ]:
structured_model = evaluator_llm.with_structured_output(EvaluationSchema)


EvaluationSchema(evaluation='approved', feedback='The tweet is concise, grammatically correct, and uses an emoji and hashtag effectively for engagement. The message is clear and motivational.', score=4.5)

In [22]:
def evaluate_tweet(state:TweetSatate):
    #prompt 
    prompt = [
        SystemMessage(content="""
        You are a strict tweet evaluator. 
        You must only respond with one of two exact strings: "approved" or "need_improvement". 
        Do not explain, justify, or output anything else. 
        Evaluation criteria: 
        - Grammar and spelling must be correct. 
        - The tweet must be clear and easy to understand. 
        - The tweet must show engagement potential (catchy or interesting). 
        - The tone must be consistent (professional, casual, or funny, but not confusing). 
        - The tweet must not exceed 300 characters. 
        - The tweet must not contain offensive, harmful, or unsafe content.
        """),
        HumanMessage(content=f"""
        Evaluate the following tweet strictly against the criteria. 
        eturn only one word: "approved" or "need_improvement".
        Tweet: {state['tweet']}
        ### respond only in structured formate
        """)
    ]
    # llms
    response=structured_model.invoke(prompt)
    
    return{
        'evaluation' : response.evaluation,
        'feedback' : response.feedback
    }

In [26]:
def optimize_tweet(state: TweetSatate):
    prompt_optimized_tweet = [
        SystemMessage(content="""
        You are a helpful assistant that optimizes tweets.
        Your task:
        - Keep the message clear, grammatically correct, and engaging.
        - Ensure the tweet is under 300 characters.
        - Optionally add 1–2 relevant emojis or hashtags if they improve engagement.
        - Do not change the core meaning of the original tweet.
        - Respond only with the optimized tweet text, no explanations.
        """),
        HumanMessage(content=f"""
        rewrite and optimize the following tweet:
        topic of the tweet - {state['topic']}
        orignal_tweet-"{state['tweet']}"
        """)
    ]
    response = optimizer_llm.invoke(prompt_optimized_tweet).content.strip()
    iteration = state['iteration'] + 1
    return {'tweet': response,'iteration':iteration}


In [ ]:
def route_evaluation(state:TweetSatate):
    if state['evaluation'] == 'approved' or state['iteration'] == state['max_iteration']:
        return 'approved'
    else:
        'need_improvement'



In [ ]:
#node
graph.add_node("genrate",genrate_tweet)
graph.add_node("evlauate",evaluate_tweet)
graph.add_node("optimize_tweet",optimize_tweet)

# make edges
graph.add_edge(START,'genrate')
graph.add_edge('genrate','evlauate')
graph.add_conditional_edges('evaluate',)